In [ ]:
// Nuget Packages
#r "nuget: MelonChart.NET, 2.*"

#r "nuget: Microsoft.Extensions.Configuration.Json, 8.*"
#r "nuget: Microsoft.SemanticKernel, 1.*"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.*"
#r "nuget: Microsoft.SemanticKernel.Core, 1.*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.*-*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.*-*"

#r "nuget: SpotifyAPI.Web, 7.*"

#r "nuget: System.Linq.Async, 6.*"


In [ ]:
// Add using statements
using System.ComponentModel;
using System.IO;
using System.Net.Http;
using System.Text.Encodings.Web;
using System.Text.Json;
using System.Text.Json.Serialization;

using MelonChart.Models;

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Memory;

using SpotifyAPI.Web;

using Kernel = Microsoft.SemanticKernel.Kernel;

In [ ]:
// Import app settings
var path = Path.Combine(Directory.GetCurrentDirectory(), "appsettings.Proxy.json");
var config = new ConfigurationBuilder().AddJsonFile(path).Build();


In [ ]:
// Azure OpenAI configurations
var endpoint = config["Azure:OpenAI:Endpoint"];
var apiKey = config["Azure:OpenAI:ApiKey"];
var deploymentName = config["Azure:OpenAI:DeploymentName"];

In [ ]:
// Build Semantic Kernel
var kernel = Kernel.CreateBuilder()
                   .AddAzureOpenAIChatCompletion(
                       deploymentName: deploymentName,
                       endpoint: endpoint,
                       apiKey: apiKey)
                   .Build();

In [ ]:
// Import prompts
var prompts = kernel.ImportPluginFromPromptDirectory("Prompts/Spotify");

In [ ]:
// Import codes
#!import Plugins/ChartMemories/SpotifyChartPlugin.cs

In [ ]:
// Import plugins
kernel.ImportPluginFromType<SpotifyChartPlugin>();

In [ ]:
// Add HttpClient instance.
var http = new HttpClient();

In [ ]:
// Add JsonSerializerOptions instance.
var jso = new JsonSerializerOptions()
{
    WriteIndented = false,
    PropertyNamingPolicy = JsonNamingPolicy.CamelCase,
    Encoder = JavaScriptEncoder.UnsafeRelaxedJsonEscaping,
    Converters = { new JsonStringEnumConverter(JsonNamingPolicy.CamelCase) },
};

In [ ]:
// User input
var question = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("지금 무슨 생각을 하고 계세요?");

Console.WriteLine($"User: {question}");

In [ ]:
// Invoke the prompt - Get Sentiments
var sentiments = await kernel.InvokeAsync<string>(
    function: prompts["GetSentiments"],
    arguments: new KernelArguments()
    {
        { "input", question }
    });

Console.WriteLine(sentiments);

In [ ]:
// Invoke the prompt - Measure Sentiments
var measured = await kernel.InvokeAsync<string>(
    function: prompts["MeasureSentiments"],
    arguments: new KernelArguments()
    {
        { "input", question },
        { "sentiments", sentiments },
    });

Console.WriteLine(measured);

In [ ]:
// Invoke the prompt - Pick Primary Sentiment
var sentiment = await kernel.InvokeAsync<string>(
    function: prompts["PickPrimarySentiment"],
    arguments: new KernelArguments()
    {
        { "input", question },
        { "measured", measured },
    });

Console.WriteLine(sentiment);

In [ ]:
// Invoke the prompt - RefineQuestion
var refined = await kernel.InvokeAsync<string>(
    function: prompts["RefineQuestion"],
    arguments: new KernelArguments()
    {
        { "input", question },
        { "sentiment", sentiment }
    });

Console.WriteLine(refined);

In [ ]:
// Invoke the plugin - Get Spotify Chart tracks
var tracks = await kernel.InvokeAsync(
    pluginName: nameof(SpotifyChartPlugin),
    functionName: nameof(SpotifyChartPlugin.GetChart), 
    arguments: new KernelArguments()
    {
        { "http", http },
        { "jso", jso }, 
    }
);

Console.WriteLine(tracks);

In [ ]:
// Invoke the plugin - Find songs
var result = await kernel.InvokeAsync<string>(
    function: prompts["FindSongs"],
    arguments: new KernelArguments()
    {
        { "input", refined },
        { "tracks", tracks }
    });

Console.WriteLine(result);


In [ ]:
// Invoke the prompt - RefineResult
var refinedTracks = await kernel.InvokeAsync<string>(
    function: prompts["RefineResult"],
    arguments: new KernelArguments()
    {
        { "input", result }
    });

Console.WriteLine(refinedTracks);

In [ ]:
// Add SpotifyClient instance.
var baseUrl = config["Azure:APIM:BaseUrl"];
var subscriptionKey = config["Azure:APIM:SubscriptionKey"];

http.DefaultRequestHeaders.Clear();
http.DefaultRequestHeaders.Add("Ocp-Apim-Subscription-Key", subscriptionKey);
var accessToken = await http.GetStringAsync($"{baseUrl}/spotify/access-token").ConfigureAwait(false);
var spotify = new SpotifyClient(accessToken);

In [ ]:
// Invoke the plugin - Genereate playlist on Spotify
var playlist = await kernel.InvokeAsync(
    pluginName: nameof(SpotifyChartPlugin),
    functionName: nameof(SpotifyChartPlugin.CreatePlaylist), 
    arguments: new KernelArguments()
    {
        { "spotify", spotify },
        { "tracks", refinedTracks },
        { "jso", jso }, 
    }
);

Console.WriteLine(playlist);